### Automação do SUAP-IFPE versão 0.1

#### Importando webdriver e abrindo SUAP

In [31]:
import pandas as pd
from datetime import datetime

arquivo = '2023-08-14_10-11_transferencia_cadeiras_CBJA.xlsx'
tombos_df = pd.read_excel(f'input/{arquivo}')
#display(tombos_df.head())
# tombos_df = tombos_df.iloc[:5]

tombos_df["tombo"] = tombos_df["tombo"].astype(int)

print(tombos_df)

autentica = pd.read_excel('ls.xlsx')
autentica['login'] = autentica['login'].astype(int)

login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

nome = 'CAFI'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

indisponivel = "indisponivel"

     tombo status consetvation  \
0   110257  Ativo          Bom   
1    39144  Ativo          Bom   
2   110256  Ativo          Bom   
3   110254  Ativo          Bom   
4   110271  Ativo          Bom   
5   110195  Ativo          Bom   
6    39118  Ativo          Bom   
7    39166  Ativo          Bom   
8   110191  Ativo          Bom   
9   110315  Ativo          Bom   
10   39034  Ativo          Bom   
11   39020  Ativo          Bom   
12   39023  Ativo          Bom   
13   39012  Ativo          Bom   
14   39010  Ativo          Bom   
15   39005  Ativo          Bom   
16   39038  Ativo          Bom   
17   39014  Ativo          Bom   
18   38989  Ativo          Bom   
19   39461  Ativo          Bom   
20   38997  Ativo          Bom   
21   39456  Ativo          Bom   
22   39455  Ativo          Bom   
23   39457  Ativo          Bom   
24  110206  Ativo          Bom   
25  110205  Ativo          Bom   
26   38987  Ativo          Bom   
27   39450  Ativo          Bom   
28  110192  At

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from webdriver_manager.firefox import GeckoDriverManager




def abrir_suap():
    
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    nav.set_page_load_timeout(60)
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [33]:
def trocar_sala():
    
    abrir_suap()
    
    for linha in tombos_df["tombo"]:
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            valor_sala_suap = nav.find_elementfind_element(By.XPATH, '//*[@id="id_sala"]').get_attribute('value')
            valor_nova_sala = '1990'

            # time.sleep(1)

            if valor_sala_suap != valor_nova_sala:
                nova_sala = Select(nav.find_elementfind_element(By.ID, 'id_sala'))
                nova_sala.select_by_value(valor_nova_sala)
                time.sleep(1)
                nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[2]/input').click()
        
        except NoSuchElementException:
            continue

    
    nav.quit()

In [34]:
def listar_carga():
    
    abrir_suap()
    
    tombo_impressao = []
    carga_impressao = []
    descricao_impressao = []    
    
    print(f'Extração iniciada as: {datetime.now()}')
    
    for linha in tombos_df["tombo"]:
        
        tombo_impressao.append(linha)
        # print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            carga = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
            carga_impressao.append(carga)
            # print(carga)

            descricao = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td[2]/span[1]").get_attribute('innerText')
            descricao_impressao.append(descricao)
            # print(descricao)
        except NoSuchElementException:
            carga_impressao.append(indisponivel)
            descricao_impressao.append(indisponivel)
            continue
            
    tabela_carga = pd.DataFrame()
    tabela_carga['tombo'] = tombo_impressao
    tabela_carga['descricao'] = descricao_impressao
    tabela_carga['carga'] = carga_impressao

    #display(tabela_carga)

    print(f'Extração finalizada as: {datetime.now()}')
    
    tabela_carga.to_excel(f'output/{data_e_hora}_lista_carga_'+arquivo, index=False, sheet_name="plan1")
        
    nav.quit()

In [35]:
def impressao_etiqueta():
    
    abrir_suap()
    
    tombo_etiqueta = []    
    empenho_impressao = []
    uasg_impressao = []
    nf_impressao = []
    dtnf_impressao = []
    qrcode_impressao = []
    
    for linha in tombos_df["tombo"]:
        
        tombo_etiqueta.append(linha)
        # print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")
            nav.find_element_byfind_element(By.LINK_TEXT, "Detalhes da Entrada").click()

            empenho = nav.find_elementfind_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
            empenho = empenho[:12]
            empenho_impressao.append(empenho)
            # print(empenho)
        
            uasg = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
            uasg_impressao.append(uasg)
            # print(uasg)

            nf = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
            nf_impressao.append(nf)
            # print(nf)

            data = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
            dtnf_impressao.append(data)
            # print(data)

            qrcode = str(linha)+" "+empenho+" "+" "+uasg+" "+nf+" "+data
            qrcode_impressao.append(qrcode)

            
        except NoSuchElementException:
            # print(indisponivel)
            empenho_impressao.append(indisponivel)
            uasg_impressao.append(indisponivel)
            nf_impressao.append(indisponivel)
            dtnf_impressao.append(indisponivel)
            qrcode_impressao.append(indisponivel)
            continue    
        
    tabela_impressao = pd.DataFrame()
    tabela_impressao['patrimonio'] = tombo_etiqueta
    tabela_impressao['empenho'] = empenho_impressao
    tabela_impressao['uasg'] = uasg_impressao
    tabela_impressao['nota fiscal'] = nf_impressao
    tabela_impressao['data nota fiscal'] = dtnf_impressao
    tabela_impressao['qrcode'] = qrcode_impressao
    
#     display(tabela_impressao)
    
    tabela_impressao.to_excel("impressao_"+arquivo, index=False, sheet_name="plan1")
        

    nav.quit()

In [36]:
def complement_item(item_list):
    
    abrir_suap()
    
    empenho = []
    ug_ne = []
    uasg = []
    nf = []
    dt_nf = []
    type_entry = []
    supplier = []
    process = []
    
    print(f'Extração iniciada as: {datetime.now()}')
    
    for linha in item_list:
        
#         print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")
            nav.find_element(By.LINK_TEXT, "Detalhes da Entrada").click()

            ne = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
            ug = ne[18:22]
            ne = ne[:12]
            empenho.append(ne)
            ug_ne.append(ug)
            # print(empenho)
        
            gest = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
            uasg.append(gest)
            # print(uasg)

            note = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
            nf.append(note)
            # print(nf)

            data = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
            dt_nf.append(data)
            # print(data)
            
            entry = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[3]/td").get_attribute('innerText')
            type_entry.append(entry)
            
            sp = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td/a").get_attribute('innerText')
            supplier.append(sp)
            
            pc = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[13]/td").get_attribute('innerText')
            process.append(pc)


            
        except NoSuchElementException:
            # print(indisponivel)
            empenho.append(indisponivel)
            ug_ne.append(indisponivel)
            uasg.append(indisponivel)
            nf.append(indisponivel)
            dt_nf.append(indisponivel)
            type_entry.append(indisponivel)
            supplier.append(indisponivel)
            process.append(indisponivel)            
            continue    
        
    table = pd.DataFrame()
    table['empenho'] = empenho
    table['ug_empenho'] = ug_ne
    table['uasg'] = uasg
    table['nt_fiscal'] = nf
    table['dt_nf'] = dt_nf
    table['tipo_entrada'] = type_entry
    table['fornecedor'] = supplier
    table['processo'] = process
    
#     display(tabela_impressao)
    print(f'Extração finalizada as: {datetime.now()}')
    table.to_excel(f'output/{data_e_hora}_{nome}.xlsx', index=False, sheet_name="plan1")
        

#     nav.quit()

In [37]:
def trocar_carga_contabil():
    
    abrir_suap()
    i = 1
    for linha in tombos_df["tombo"]:
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/select').click()
            nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/select').send_keys('BEL')
            nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/select').send_keys(Keys.ENTER)
            time.sleep(2)
            nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/span').click()
#             nav.find_element_by_link_text("Alterar").click()
            time.sleep(2)    
            
            local = nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]').text
#             print(f'tombo: {linha} - carga contabil: {local} - {i}')
            i+=1
            
        except NoSuchElementException:
            local = nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]').text
            print(f'tombo: {linha} - carga contabil: {local} - {i}')
            i+=1
            continue

    
    nav.quit()

In [38]:
def set_requisition_pat(tombos):
#     abrir_suap()
    li = 0
    origin = 'Carlos Eduardo Cabral'
    destin = 'Marcos Antônio Germano'
    
    nav.get(f"https://suap.ifpe.edu.br/patrimonio/requisitar_saida/")
    nav.find_element(By.XPATH, '//html/body/div[1]/div[3]/form/div[1]/fieldset/div[1]/div/input[1]').click()
    nav.find_element(By.XPATH, '//html/body/div[1]/div[3]/form/div[1]/fieldset/div[1]/div/input[1]').send_keys(origin)
    time.sleep(4)
    nav.find_element(By.XPATH, '//html/body/div[1]/div[3]/form/div[1]/fieldset/div[1]/div/input[1]').send_keys(Keys.ENTER)
    
    
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/fieldset/div[2]/div/input[1]').click()
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/fieldset/div[2]/div/input[1]').send_keys(destin)
    time.sleep(4)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/fieldset/div[2]/div/input[1]').send_keys(Keys.ENTER)
    
    
    for tombo in tombos:
        print(tombo)
        try:
            if li == 0:
                
                nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li/input').send_keys(f'{tombo} ')
                time.sleep(2)
                nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li/input').send_keys(Keys.ENTER)
                li += 2
            else:
                nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li[{li}]/input').send_keys(f'{tombo} ')
                time.sleep(2)
                nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li[{li}]/input').send_keys(Keys.ENTER)
                li += 1
                
            """
            /html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li/input
            /html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li[2]/input
            /html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li[3]/input
            
            /html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li/input
            /html/body/div[1]/div[3]/form/div[1]/fieldset/div[5]/div/ul/li[2]/input
            """
        except ElementNotInteractableException:
            print('elemento não encontrado')
            continue

In [39]:
trocar_carga_contabil()


tombo: 110257 - carga contabil: BELO JARDIM - 1
tombo: 39038 - carga contabil: BELO JARDIM - 17
